In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [3]:
import os
import pandas as pd

excel_path = "content\Results.xlsx"
images_dir = "content\sem_images"

# 1. Check Excel file and sheets
xls = pd.ExcelFile(excel_path)
print("Sheet names in Excel:")
print(xls.sheet_names)

print("\n---\n")

# 2. Check images folder
print("Images directory exists?", os.path.exists(images_dir))
print("Files in images_dir:")
print(os.listdir(images_dir))


Sheet names in Excel:
['results_1', 'result_2', 'result_3', 'result_4', 'result_5', 'result_6', 'result_7', 'result_8', 'result_9']

---

Images directory exists? True
Files in images_dir:
['1_001.tif', '1_002.tif', '1_003.tif', '1_004.tif', '2_001.tif', '2_002.tif', '3_001.tif', '3_002.tif', '3_003.tif', '4_001.tif', '4_002.tif', '4_003.tif', '4_004.tif', '4_005.tif', '5_001.tif', '5_002.tif', '5_003.tif', '5_004.tif', '6_001.tif', '6_002.tif', '6_003.tif', '6_004.tif', '6_005.tif', '6_006.tif', '6_007.tif', '6_008.tif', '7_001.tif', '7_002.tif', '7_003.tif', '7_004.tif', '7_005.tif', '8_001.tif', '8_002.tif', '9_001.tif', '9_002.tif', '9_003.tif']


In [6]:
sheet_to_check = xls.sheet_names[0]  # or "3" or "Bi-0.02" etc

print("Checking sheet:", sheet_to_check)

df_sample = pd.read_excel(excel_path, sheet_name=sheet_to_check)
print("\nColumns in this sheet:")
print(df_sample.columns)

print("\nFirst 5 rows:")
print(df_sample.head())

# Clean column names and check again
df_sample.columns = [str(c).strip() for c in df_sample.columns]
print("\nCleaned column names:")
print(df_sample.columns)

if "Length" in df_sample.columns:
    print("\nMean Length for this sheet:", df_sample["Length"].mean())
else:
    print("\n❌ 'Length' column not found after cleaning.")


Checking sheet: results_1

Columns in this sheet:
Index([' ', 'Area', 'Mean', 'Min', 'Max', 'Angle', 'Length'], dtype='object')

First 5 rows:
       Area     Mean     Min      Max    Angle  Length
0  1  0.084  109.031  83.894  150.617  107.700   2.852
1  2  0.109  101.872  75.000  155.279   81.995   3.736
2  3  0.072   95.843  66.000  117.514  140.711   2.465
3  4  0.093   99.980  82.553  139.890   60.642   3.183
4  5  0.101  106.292  57.000  152.000  101.497   3.480

Cleaned column names:
Index(['', 'Area', 'Mean', 'Min', 'Max', 'Angle', 'Length'], dtype='object')

Mean Length for this sheet: 2.8886422413793102


In [9]:
print("Testing filename matching logic...\n")

for sheet in xls.sheet_names:
    sample_id = (str(sheet).strip().split("_")[-1])
    print(f"Sheet: '{sheet}'  -> sample_id used for matching: '{sample_id}'")

    # list images that start with that sample_id + "_"
    matched = []
    for fname in os.listdir(images_dir):
        if fname.lower().endswith((".tif", ".tiff", ".png", ".jpg", ".jpeg")):
            if fname.startswith(sample_id + "_"):
                matched.append(fname)

    print("  Matched images:", matched)
    print("  Count:", len(matched))
    print("---")


Testing filename matching logic...

Sheet: 'results_1'  -> sample_id used for matching: '1'
  Matched images: ['1_004.tif', '1_003.tif', '1_001.tif', '1_002.tif']
  Count: 4
---
Sheet: 'result_2'  -> sample_id used for matching: '2'
  Matched images: ['2_002.tif', '2_001.tif']
  Count: 2
---
Sheet: 'result_3'  -> sample_id used for matching: '3'
  Matched images: ['3_002.tif', '3_003.tif', '3_001.tif']
  Count: 3
---
Sheet: 'result_4'  -> sample_id used for matching: '4'
  Matched images: ['4_005.tif', '4_001.tif', '4_002.tif', '4_004.tif', '4_003.tif']
  Count: 5
---
Sheet: 'result_5'  -> sample_id used for matching: '5'
  Matched images: ['5_002.tif', '5_003.tif', '5_001.tif', '5_004.tif']
  Count: 4
---
Sheet: 'result_6'  -> sample_id used for matching: '6'
  Matched images: ['6_004.tif', '6_002.tif', '6_003.tif', '6_001.tif', '6_006.tif', '6_007.tif', '6_008.tif', '6_005.tif']
  Count: 8
---
Sheet: 'result_7'  -> sample_id used for matching: '7'
  Matched images: ['7_005.tif', '7_0